# HA 3

In [1]:
!pip install sympy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import sympy
import random
import hashlib
import math 

### Problem 1 (1 point)

Diffie–Hellman key exchange protocol is one of the earliest practical examples of public key exchange implemented within the field of cryptography. Published in 1976 by Diffie and Hellman, this is the earliest publicly known work that proposed the idea of a private key and a corresponding public key. 

1) Implement function to generate common secret key within multiplicative group of given Finite field with known generator. 

*Note. You can assume that all the numbers are small, for example, less than 1000.*

In [16]:
def generate_secret(P, G, a, b):
  # key share
  x = int(pow(G, a, P)) 

  # key share
  y = int(pow(G, b, P)) 
     
  # generated secret key
  ka = int(pow(y, a, P))
     
  # generated secret key
  kb = int(pow(x, b, P))
  
  return ka, kb

2) Test your solution in GF(17) with generator g=11. Bobs' open key B=11, Alice private key is a=7

In [22]:
P = 17
G = 11
a = 7 #Alice private key
b = sympy.randprime(3, P) #random Bob's private key selected
print(b)
print(generate_secret(P, G, a, b))

13
(12, 12)


### Problem 2 (3 points)

El Gamal protocol is widely used in cryptography. In this task we will ask you to implement your own El-Gamal encryption scheme on Python.

1) Implement function for generating keys. The function must generate big random prime number (problem of generating big prime numbers was discussed within the lectures). (1 point)

*Note. You can assume that all the numbers are small, for example, less than $2^{32}$. But you **must** use a primality test as a part of the function to get a full score.*

In [3]:
# p - big prime number
# random x (private key); gcd(x, p) = 1; x < p
# random number g; g < p
# y = g^x mod p
# public key (y, g, p)
# private key (x, g, p)

def gcd(a, b):
    if a < b:
        return gcd(b, a)
    elif a % b == 0:
        return b
    else:
        return gcd(b, a % b)
    
def generate_key(p):
    key = random.randint(math.pow(10, 10), p)
    while gcd(p, key) != 1:
        key = random.randint(math.pow(10, 10), p)
        
    return key

def computation(g, x, p):
    t = 1
    y = g
 
    while x > 0:
        if x % 2 != 0:
            t = (t * y) % p
        y = (y * y) % p
        x = int(x / 2)
 
    return t % p

def main():
    p = sympy.randprime(math.pow(10, 10), math.pow(10, 15))
    g = random.randint(2, p)
    private_key = generate_key(p)
    y = computation(g, private_key, p)
    print("private key : x =", private_key, "g =",  g, "p =", p)
    print("public key : y =", y, "g =", g, "p =", p)
    
if __name__ == '__main__':
    main()

private key : x = 53679172022427 g = 102028091956141 p = 160719367789403
public key : y = 11671582068884 g = 102028091956141 p = 160719367789403


2) Implement functions that realize the encryption and decryption in El Gamal protocol. (1 points)

In [4]:
# random k; k < p - 1; gcd(k, p) = 1
# a = g^k mod p
# b = g^(x*k) mod p
# message: (a, b * M)
def encryption(msg, g, p, y):
    en_msg = []
        
    k = generate_key(p)
    a = computation(g, k, p)
    b = computation(y, k, p)
    
    for i in range(0, len(msg)):
        en_msg.append(msg[i])
        
    for i in range(0, len(en_msg)):
        en_msg[i] = b * ord(en_msg[i])
    
    return a, en_msg

# b' = a^x = g^(k*x) mod p
# (b * M) / b' = M (b = b')
def decryption(en_msg, a, x, p):
    decr_msg = []
    
    b_ = computation(a, x, p)
    for i in range(0, len(en_msg)):
        decr_msg.append(chr(int(en_msg[i] / b_)))
         
    return ''. join(decr_msg)

def main():
    a, en_msg = encryption(msg, g, p, y)
    print("Message after decryption: ", decryption(en_msg, a, x, p))

if __name__ == '__main__':
    p = sympy.randprime(math.pow(10, 10), math.pow(10, 15))
    g = random.randint(2, p)
    x = generate_key(p)
    y = computation(g, x, p)
    msg = 'Hello, world!'
    main()    

Message after decryption:  Hello, world!


3) Calculate Hash of your name by SHA-1 and test El Gamal encryption/decryption functions on its 16-bit prefix. (1 points)

In [5]:
sha1 = hashlib.sha1(b"Inna").hexdigest()
print("Hash: ", sha1)

prefix = sha1[0:4]
p = sympy.randprime(math.pow(10, 10), math.pow(10, 15))
g = random.randint(2, p)
x = generate_key(p)
y = computation(g, x, p)
a, en_msg = encryption(prefix, g, p, y)
print("Prefix after decryption: ", decryption(en_msg, a, x, p))

Hash:  4d717d507843a696c39d3d1e7857e2155a31f32d
Prefix after decryption:  4d71


### Problem 3 (4 points)

Elliptic curves due to their efficient hardware realization widely used in modern secure communication channels. The main thing that lies inside their cryptographic hardness is that we can break them only by greed search over all group points. In this task, we will ask you to write Python function that returns all group elements of a certain elliptic curve over a finite field 

1) Write a python function that list all points of elliptic curve $y^2=x^3+7$ over $F_{127}$ (1 point)

*Note. $127 = 2^7-1$ is the fourth Mersenne prime.*

In [23]:
INFINITY_PNT = None

class EllipticCurve:
    def __init__(self, a, b, p):
        self.a = a
        self.b = b
        self.p = p
        self.points = []
        self.findPoints()

    def findPoints(self):
        self.points.append(INFINITY_PNT)
        for x in range(self.p):
            for y in range(self.p):
                if self.isEqual(y * y, x * x * x + self.a * x + self.b):
                    self.points.append((x,y))


    def addition(self, p1, p2):
        if p1 == INFINITY_PNT:
            return p2
        if p2 == INFINITY_PNT:
            return p1

        x1, y1, x2, y2 = p1[0], p1[1], p2[0], p2[1]
  
        if self.isEqual(x1, x2) and self.isEqual(y1, -y2):
            return INFINITY_PNT

        if self.isEqual(x1, x2) and self.isEqual(y1, y2):
            u = self.modP((3 * x1 * x1 + self.a) * self.inverse(2 * y1))
        else:
            u = self.modP((y1 - y2) * self.inverse(x1 - x2))

        v = self.modP(y1 - u * x1)
        x3 = self.modP(u * u - x1 - x2)
        y3 = self.modP(-u * x3 - v)

        return (x3, y3)

    def countPoints(self):
        return len(self.points)

    def printPoints(self):
        print(self.points)

    def modP(self, x):
        return x % self.p

    def isEqual(self, x, y):
        return self.modP(x - y) == 0

    def inverse(self, x):
        for y in range(self.p):
            if self.isEqual(x * y, 1):
                return y
        return None

p = 127
ec = EllipticCurve(0, 7, p)
ec.printPoints()

[None, (1, 32), (1, 95), (2, 53), (2, 74), (3, 62), (3, 65), (4, 43), (4, 84), (8, 30), (8, 97), (11, 24), (11, 103), (12, 46), (12, 81), (14, 46), (14, 81), (17, 27), (17, 100), (18, 39), (18, 88), (19, 32), (19, 95), (21, 39), (21, 88), (24, 49), (24, 78), (25, 30), (25, 97), (28, 49), (28, 78), (32, 3), (32, 124), (34, 24), (34, 103), (38, 53), (38, 74), (39, 12), (39, 115), (41, 27), (41, 100), (45, 33), (45, 94), (46, 51), (46, 76), (47, 43), (47, 84), (51, 18), (51, 109), (52, 36), (52, 91), (57, 62), (57, 65), (58, 38), (58, 89), (60, 19), (60, 108), (67, 62), (67, 65), (69, 27), (69, 100), (70, 19), (70, 108), (71, 63), (71, 64), (72, 16), (72, 111), (75, 49), (75, 78), (76, 43), (76, 84), (78, 50), (78, 77), (79, 63), (79, 64), (80, 18), (80, 109), (82, 24), (82, 103), (84, 16), (84, 111), (85, 50), (85, 77), (86, 38), (86, 89), (87, 53), (87, 74), (88, 39), (88, 88), (91, 50), (91, 77), (93, 33), (93, 94), (94, 30), (94, 97), (96, 51), (96, 76), (98, 16), (98, 111), (99, 36),

2) Compare the number of points with Hasse’s estimate $|N-(q+1)|\leq 2{\sqrt  {q}}$. (1 point)

In [24]:
N = ec.countPoints()
q = 127
print("Number of points = ", N)
if abs(N - (q + 1)) <= 2 * math.sqrt(q):
  print("Hasse's theorem is approved")
else:
  print("Hasse's theorem is declined")

Number of points =  127
Hasse's theorem is approved


3) Prove that the point
$A = (19, 32)$ belongs to the elliptic curve and construct a sequence of $B_n = nA, n = 1, ..., 100$. (2 points)

In [8]:
if (19, 32) in ec.points:
  print("Point 𝐴=(19,32) belongs to the elliptic curve")
  print(ec.points.index((19, 32)))
else:
  print("Point 𝐴=(19,32) does not belong to the elliptic curve")

Point 𝐴=(19,32) belongs to the elliptic curve
21


In [9]:
seq = []
a = ec.points[21]
for n in range(1, 101):
  seq.append(a)
  a = ec.addition(a, ec.points[21])
print(seq)
print(len(seq))

[(19, 32), (11, 24), (98, 16), (123, 18), (79, 64), (110, 38), (58, 38), (38, 53), (67, 62), (91, 50), (25, 30), (69, 27), (86, 89), (41, 27), (103, 91), (122, 3), (60, 19), (24, 78), (1, 32), (107, 95), (75, 78), (109, 12), (21, 39), (4, 84), (2, 53), (17, 100), (104, 64), (76, 84), (96, 76), (116, 94), (99, 91), (71, 63), (87, 74), (94, 30), (45, 94), (101, 46), (106, 12), (34, 24), (28, 49), (82, 103), (124, 19), (14, 46), (112, 76), (57, 62), (8, 97), (52, 36), (51, 18), (18, 39), (12, 46), (100, 3), (80, 109), (47, 43), (3, 65), (78, 50), (46, 76), (88, 39), (72, 16), (70, 108), (39, 115), (84, 111), (32, 3), (93, 94), (85, 50), (85, 77), (93, 33), (32, 124), (84, 16), (39, 12), (70, 19), (72, 111), (88, 88), (46, 51), (78, 77), (3, 62), (47, 84), (80, 18), (100, 124), (12, 81), (18, 88), (51, 109), (52, 91), (8, 30), (57, 65), (112, 51), (14, 81), (124, 108), (82, 24), (28, 78), (34, 103), (106, 115), (101, 81), (45, 33), (94, 97), (87, 53), (71, 64), (99, 36), (116, 33), (96, 51

### Problem 4 (2 points)

Let $p = 601$, $q = 7$, $e = 1463$ be the setup of the RSA algorithm. Compute $d$, sign the plane message $m = 58$ and check the signature.

In [13]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    else:
        return x % m

In [18]:
p = 601
q = 7
e = 1463 #public key
m = 58
n = p * q
phi = (p - 1) * (q - 1)
# d = e^-1 mod (p-1)*(q-1) 
# pow(a,-b,c) == pow((a^-1) mod c, b, c)
d = pow(modinv(e, phi), 1, phi) #private key
print("Decryption key =", d)

# digital signature S=m^e mod n
# sends (M, S)
# m1=S^d mod n 

# Signature 
S = (m**e) % n
print("Signature =", S)

m1 = (S**d) % n
if m == m1:
    print("Signature is valid")

Decryption key = 1127
Signature = 2937
Signature is valid


### Problem 5 (4* points)

*bonus problem

Consider an elliptic curve $y^2 = x^3 - 2 x - 1$.

1) (1 point) Plot the curve for $(x, y) \in (-19, 19)$. Plot $$\{(x \mod 19, y \mod 19) \; | \;  (x, y) \in (-19 k, 19 k)^2 \; and \; y^2 = x^3 - 2 x - 1\}$$ for $k = 1, \dots, 5$. 

2) (1 point) Scatter all the integer points of curve $y^2 = x^3 - 2 x - 1$ $\mod 19$. How many points are not in $$\{(x \mod 19, y \mod 19) \; | \;  (x, y) \in (-19 \cdot 5, 19  \cdot 5)^2 \; and \; y^2 = x^3 - 2 x - 1\}?$$ 

3) (2 points) Choose integer examples for all three elliptic curve $y^2 = x^3 - 2 x - 1$ addition cases and illustrate them with (straight) lines.